# Profiling of terrain corrections

<hr style="border:2px solid blue"> </hr>

### Install Dependencies and Import

Additional dependecies: `sarsen`, `snakeviz`

In [ ]:
!pip install -q sarsen snakeviz

In [ ]:
%load_ext snakeviz

In [ ]:
import os
import tempfile

import adlfs
import planetary_computer
import pystac_client

# enable the `.rio` accessor
import rioxarray  # noqa: F401
import stackstac
from sarsen.apps import terrain_correction

### Processing definitions

In [ ]:
# create a temporary directory where to store downloaded data
tmp_dir = tempfile.gettempdir()

# DEM path
dem_path = os.path.join(tmp_dir, "South-of-Redmond-10m.tif")

# path to Sentinel-1 input product in the Planetary Computer
product_folder = "GRD/2021/12/17/IW/DV/S1B_IW_GRDH_1SDV_20211217T141304_20211217T141329_030066_039705_9048"  # noqa: E501

# band to be processed
measurement_group = "IW/VV"

tmp_dir

#### Area of interest definition: South-of-Redmond (Seattle, US)

In [ ]:
lon, lat = [-121.95, 47.04]
buffer = 0.2
bbox = [lon - buffer, lat - buffer, lon + buffer, lat + buffer]

#### DEMs discovery

In [ ]:
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1"
)
search = catalog.search(collections="3dep-seamless", bbox=bbox)
items = list(search.get_items())

In [ ]:
# select DEMs with resolution 10 meters
items_high_res = [
    planetary_computer.sign(item).to_dict()
    for item in items
    if item.properties["gsd"] == 10
]

dem_raster_all = stackstac.stack(items_high_res, bounds=bbox).squeeze()
dem_raster_all

#### DEMs average along the time dimension

In [ ]:
dem_raster_geo = dem_raster_all.compute()
if "time" in dem_raster_geo.dims:
    dem_raster_geo = dem_raster_geo.mean("time")
_ = dem_raster_geo.rio.set_crs(dem_raster_all.rio.crs)

#### Convert the DEM in UTM coordinates

In [ ]:
# find the UTM zone and project in UTM
t_srs = dem_raster_geo.rio.estimate_utm_crs()
dem_raster = dem_raster_geo.rio.reproject(t_srs, resolution=(5, 5))

# crop DEM to our area of interest and save it
dem_corners = dict(x=slice(565000, 594000), y=slice(5220000, 5190000))


dem_raster = dem_raster.sel(**dem_corners)
dem_raster.rio.to_raster(dem_path)
dem_raster

### Define GRD parameters

In [ ]:
grd_account_name = "sentinel1euwest"
grd_storage_container = "s1-grd"
grd_product_folder = f"{grd_storage_container}/{product_folder}"
grd_local_path = os.path.join(tmp_dir, product_folder)

### Retrieve Sentinel-1 GRD

In [ ]:
grd_token = planetary_computer.sas.get_token(
    grd_account_name, grd_storage_container
).token
grd_fs = adlfs.AzureBlobFileSystem(grd_account_name, credential=grd_token)
grd_fs.ls(f"{grd_product_folder}/manifest.safe")

In [ ]:
grd_fs.get(grd_product_folder, grd_local_path, recursive=True)
!ls -d {grd_local_path}

### Profiling

#### GTC

In [ ]:
%%snakeviz
terrain_correction(
    product_urlpath=grd_local_path,
    measurement_group=measurement_group,
    dem_urlpath=dem_path,
    output_urlpath=os.path.join(
        tmp_dir, os.path.basename(product_folder) + ".10m.GTC.tif"
    ),
)

#### RTC

##### Nearest neighbour

In [ ]:
%%snakeviz
terrain_correction(
    grd_local_path,
    measurement_group=measurement_group,
    dem_urlpath=dem_path,
    correct_radiometry="gamma_nearest",
    output_urlpath=os.path.join(
        tmp_dir, os.path.basename(product_folder) + ".10m.RTC.tif"
    ),
    grouping_area_factor=(3, 3),
)

##### Bilinear

In [ ]:
%%snakeviz
terrain_correction(
    grd_local_path,
    measurement_group=measurement_group,
    dem_urlpath=dem_path,
    correct_radiometry="gamma_bilinear",
    output_urlpath=os.path.join(
        tmp_dir, os.path.basename(product_folder) + ".10m.RTC.tif"
    ),
    grouping_area_factor=(3, 3),
)